### <Center> Лабораторна робота №6. <br> Ідентифікація користувача за допомогою логістичної регресії

In [1]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

Спочатку налаштуємо доступ до даних на google drive (якщо ви відкриваєте блокнот в google colab, а не на PC) шляхом монтування google drive

In [2]:
# шлях до папки з даними на моєму github, відредагуйте згідно вашого випадку
data_folder = "https://raw.githubusercontent.com/Kenobi-Knobs/DataScienceSSUCourse/master/Lab%203/data/" 

### 1. Завантаження і перетворення даних
Дані можна самостійно завантажити за посиланням [сторінка](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2). Однак можна цього не робити, оскільки дані вже завантажені для проведення лабораторної роботи

In [3]:
# завантажимо навчальну і тестову вибірки
train_df = pd.read_csv(data_folder+'/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv(data_folder+'/test_sessions.csv',
                      index_col='session_id')

# приведемо колонку time1, ..., time10 до часового формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# відсортуємо дані за часом
train_df = train_df.sort_values(by='time1')

# подивимося на заголовок навчальної вибірки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В навчальній вибірці містяться наступні ознаки:
    - site1 – індекс першого відвідування сайту в сесії
    - time1 – час відвідування першого сайту в сесії
    - ...
    - site10 – індекс 10-го відвідування сайту в сесії
    - time10 – час відвідування 10-го сайту в сесії
    - target – цільова змінна, 1 для сесій Еліс, 0 для сесій інших користувачів
    
Сесії користувачів виділені таким чимном, щоб вони не можут бути довші півгодини чи 10 сайтів. Тобто сесія вважається закінченою або коли користувач відвідав 10 сайтів підряд або коли сесія зайняла за часом більше 30 хвилин.

В таблиці зустрічаються пропущені значення, це значить, що сесія містить менше, ніж 10 сайтів. Замінимо пропущені значення нулями і приведемо ознаки до цільового типу. Також заванатажимо словник сайтів і подивимося, як він виглядає:

In [22]:
# приведемо колонки site1, ..., site10 до цілочислового формату і замінимо пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

import requests
URL = "https://raw.githubusercontent.com/Kenobi-Knobs/DataScienceSSUCourse/master/Lab%203/data/site_dic.pkl"
response = requests.get(URL)
input_file = open("site_dic.pkl", "wb").write(response.content)

# завантажимо словник сайтів
with open(r"site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словника сайтів
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всього сайтів:', sites_dict_df.shape[0])
sites_dict_df.head()

всього сайтів: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Виділимо цільову змінну і об'єднаємо вибірки, щоб разом привести їх до розрідженого формату.

In [23]:
# наша цільова змінна
y_train = train_df['target']

# об'єднана таблиця вхідних даних
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# індекс, за яким будемо відокремлювати навчальну вибірку від тестової
idx_split = train_df.shape[0]

Для самої першої моделі ми використовуємо лише відвідувані сайти в сесіях (але не будемо звертати увагу на часові ознаки). В основі такого вибору даних для моделей лежить така ідея: * у Еліс є свої улюблені сайти, і якщо ви ще побачите ці сайти в сесіях, тим вище ймовірність, що це сесія Еліс і навпаки. *

Підготуємо дані, з усієї таблиці виберемо лише ознаки `site1, site2, ..., site10`. Нагадуємо, що пропущені значення замінені нулем. Ось як виглядатимуть перші рядки таблиць:

In [24]:
# таблиця з індексами відвіданих сайтів в сесії
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сесії представляють собою послідовність індексів сайтів і дані в такому вигляді невдалі для лінійних методів. Відповідно до нашої гіпотези (у Еліс є улюблені сайти) необхідно перетворити цю таблицю таким чином, щоб кожен можливий веб-сайт відповідав своєму окремому призначенню (колонка), а його значення зростало за кількістю відвідувачів цього веб-сайту в сесіях. Це робиться в два рядки:

In [25]:
from scipy.sparse import csr_matrix

In [26]:
csr_matrix?

In [27]:
# послідовність з індексами
sites_flatten = full_sites.values.flatten()

# шкана матриця
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

Ще один плюс використання розріджених матриць у тому, що для них є спеціальні реалізації як матричних операцій, так і алгоритми машинного навчання, що дозволяє істотно прискорити операції за рахунок особливостей структур даних. Це стосується і логістичної регресії. Ось тепер у нас все готове для побудови наших перших моделей.

### 2. Побудова першої моделі

Отже, у нас є алгоритм та дані для нього, побудуйте нашу першу модель, використовуючи реалізацію [логістичної регресії] (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) з пакета ` sklearn` з параметрами за замовчуванням. Перші 90% даних будемо використовувати для навчання (навчальна вибірка, відсортована за часом), а також 10% для перевірки якості (validation).

**Напишіть просту функцію, яка поверне якість моделей на вкладеній вибірці, і навчіть наш перший класифікатор**.

In [30]:
from sklearn.model_selection import train_test_split

def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – вибірка
    ratio – у якому співвідношенні поділити вибірку
    C, seed – коефіцієт регуляризації і random_state 
              логістичної регресії
    '''
    
    X_train, X_test, y_tr, y_test = train_test_split(X, y, test_size=1 - ratio, random_state=seed)
    logreg = LogisticRegression(random_state=seed, C=C, n_jobs=-1).fit(X_train,y_tr)
    probs = logreg.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, probs)
    return auc, logreg

**Подивіться, який отримано ROC AUC на відкладеній вибірці.**

In [31]:
train_matrix = full_sites_sparse[:idx_split, :]
auc, logreg = get_auc_lr_valid(train_matrix, y_train)
print(auc)

0.9642971215894406


Будем вважати цю модель нашою першою відправною точкою (базовий рівень). Для побудови моделей для прогнозування на тестовій вибірці ** необхідно навчити модель заново вже на всій навчальній вибірці ** (покищо наша модель навчилася лише на частині даних), що підвищує її узагальнюючу здатність:

In [32]:
# функція для запису прогнозів в файлі
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Навчіть модель на всій вибірці, зробіть прогноз для тестової вибірки і покажіть результат**.

In [33]:
test_matrix = full_sites_sparse[idx_split:, :]
print(logreg.predict(test_matrix))
print(logreg.predict_proba(test_matrix))

[0 0 0 ... 0 0 0]
[[9.97576115e-01 2.42388544e-03]
 [9.99999997e-01 3.02522238e-09]
 [9.99999989e-01 1.05567843e-08]
 ...
 [9.90792061e-01 9.20793865e-03]
 [9.99586001e-01 4.13998545e-04]
 [9.99986179e-01 1.38209328e-05]]


### 3. Покращення моделі, побудова нових ознак

Створіть таку ознаку, яка буде представлят собою число формату ГГГГММ від тої дати, коли відбувалась сесія, наприклад 201407 -- 2014 рік і 7 месяц. Таким чином, ми будемо враховувати помісячний [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь період наданих даних.

In [34]:
extra_features_sm = pd.DataFrame(index = full_df.index)
extra_features_sm['start_month'] = full_df['time1'].apply(lambda time: time.year *100 + time.month)
extra_features_sm.head()

,start_month
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


Додайте нову ознаку, попередньо нормалізуючи її за допомогою `StandardScaler`, і знову підрахуйте ROC AUC на відкладеній вибірці.

In [35]:
normalized_sm = StandardScaler().fit_transform(extra_features_sm.values.reshape(-1,1))
full_matrix_with_sm = csr_matrix(hstack((full_sites_sparse, normalized_sm)))
auc, logreg = get_auc_lr_valid(full_matrix_with_sm[:idx_split, :], y_train.values.flatten())
print(auc)

0.9687660793114546


**Додайте дві нові ознаки: start_hour і morning.**

Ознака `start_hour` - це час у якому почалася сесія (від 0 до 23), а бінарна оознака` morning` рівна 1, якщо сесія почалася вранці і 0, якщо сесія почалася пізніше (будемо вважати, що це ранок, якщо `start_hour рівний` 11 або менше).

**Підрахуйте RUC AUC на відкладеній вибірці для вибірки з:**
- сайтами, `start_month` і` start_hour`
- сайтами, `start_month` і` morning`
- сайтами, `start_month`,` start_hour` і `morning`

In [36]:
extra_features_sm['start_hour'] = full_df['time1'].apply(lambda time: time.hour)
extra_features_sm['morning'] = full_df['time1'].apply(lambda time: 1 if time.hour <= 11 else 0)

normalized_sh = StandardScaler().fit_transform(extra_features_sm['start_hour'].values.reshape(-1,1))
normalized_m = StandardScaler().fit_transform(extra_features_sm['morning'].values.reshape(-1,1))

full_matrix_with_sm_sh = csr_matrix(hstack((full_matrix_with_sm, normalized_sh)))
full_matrix_with_sm_m = csr_matrix(hstack((full_matrix_with_sm, normalized_m)))
full_matrix_with_sm_sh_m = csr_matrix(hstack((full_matrix_with_sm_sh, normalized_m)))

auc, logreg = get_auc_lr_valid(full_matrix_with_sm_sh[:idx_split, :], y_train)
print(auc)

auc, logreg = get_auc_lr_valid(full_matrix_with_sm_m[:idx_split, :], y_train)
print(auc)

auc, logreg = get_auc_lr_valid(full_matrix_with_sm_sh_m[:idx_split, :], y_train)
print(auc)

0.9800435018462628
0.9810718796274382
0.9827559966013965


### 4. Підбір коефіцієнта регуляризації

Отже, ми ввели ознаки, які покращують якість нашої моделі у порівнянні з першим бейслайном. Чи можемо ми домогтися більшого значення метрики? Після того, як ми сформували навчальну та тестову вибірки, майже завжди має сенс підібрати оптимальні гіперпараметри - характеристики моделі, які не змінюються під час навчання. Наприклад, ви вивчали вирішальні дерева, глибина дерева це гіперпараметр, а ознака, за якому відбувається розгалуження і її значення - ні. У використовуваної нами логістичної регресії ваги кожної ознаки змінюються і під час навчання знаходяться їх оптимальні значення, а коефіцієнт регуляризації залишається постійним. Це той гіперпараметр, який ми зараз будемо оптимізувати.

Порахуйте якість на відкладеній вибірці з коефіцієнтом регуляризації, який за замовчуванням `C = 1 ':

In [37]:
auc, logreg = get_auc_lr_valid(train_matrix, y_train)
print(auc)

0.9642971215894406


Постараємося побити цей результат за рахунок оптимізації коефіцієнта регуляризації. Візьмемо набір можливих значень C і для кожного з них порахуємо значення метрики на відкладеної вибірці.

Знайдіть `C` з` np.logspace (-3, 1, 10) `, при якому ROC AUC на відкладеної вибірці максимальний.

In [38]:
def compare(c_value):
  auc, model = get_auc_lr_valid(train_matrix, y_train, C = c_value)
  return auc

c_values = np.logspace (-3, 1, 10)
optimal_c = max(c_values, key = compare)
print(optimal_c) 

3.593813663804626


Нарешті, навчіть модель зі знайденим оптимальним значенням коефіцієнта регуляризації і з побудованими ознаками `start_hour`,` start_month` і `morning`. 

In [39]:
auc, logreg = get_auc_lr_valid(full_matrix_with_sm_sh_m[:idx_split, :], y_train, C = 3.59)
print(auc)

0.9833480294312924
